# Capítulo 4 - Uso de JayDeBeApi para consultar Bases de Datos
___
## Conexión
___
Para conectar con una base de datos se puede usar la librería `JayDeBeApi`:

In [ ]:
! pip install JayDeBeApi

También hay que asegurarse de que la variable de entorno JAVA_HOME tiene la carpeta donde se ha instalado un jdk de Java
1. Descargar e instalar Java jdk 1.11: https://www.oracle.com/java/technologies/javase/jdk11-archive-downloads.html > "Java SE Development Kit" > {sistema operativo. Ej.: Windows} > "x64 installer"
2. Añadir la variable de entorno JAVA_HOME = `C:\Program Files\Java\jdk{version}` (se sustituye `{version}` por la versión instalada. Ej.: `1.11.0_281`)

Importamos el módulo y creamos la siguiente clase que nos facilita hacer querys a la BBDD:

In [ ]:
import jaydebeapi

class Database(object):

    def __init__(self, java_class_name, url, driver_args, jars):
        self.jaydebeapi_args = [java_class_name, url, driver_args, jars]

    def query(self, sql):
        with jaydebeapi.connect(*self.jaydebeapi_args) as connection:
            with connection.cursor() as cursor:
                cursor.execute(sql)
                if cursor.description:
                    headers = [x[0] for x in cursor.description]
                    rows = cursor.fetchall()
                    return [dict(zip(headers, row)) for row in rows]

Creamos una base de datos [SQLite3](https://es.wikipedia.org/wiki/SQLite). A diferencia de las bases de datos tipo servidor, este tipo de BBDD guarda todo el contenido en un archivo. La cadena de conexión JDBC es `jdbc:sqlite:{path}` donde "path" es la ruta donde se creará dicho archivo (si no existe todavía)

In [ ]:
db = Database("org.sqlite.JDBC", "jdbc:sqlite:MySQLite.db", ["", ""], "sqlite-jdbc-3.36.0.3.jar")  # MySQLite.db es la ruta donde se creará la BBDD

db.query("select 1")

## Operaciones
___
### Create table

In [ ]:
db.query("""\
    CREATE TABLE IF NOT EXISTS tabla1 (
        contact_id INTEGER PRIMARY KEY,
        first_name TEXT    NOT NULL,
        last_name  TEXT,
        phone      TEXT    NOT NULL CHECK (length(phone) >= 9)
        );
    """)

## Insert

In [ ]:
db.query("""\
    INSERT INTO tabla1 (first_name, last_name, phone) VALUES('Alejandro', 'Manzanero', '987654321');
    """)

db.query("""\
    INSERT INTO tabla1 (first_name, last_name, phone)
    VALUES('Elon', 'Musk', '987654321');
    """)

## Select
Cuando la query es de tipo "select" la respuesta de `db.query` será una lista de diccionarios, donde las keys equivalen a las columnas de la tabla. En cualquier otro caso la respuesta es `None`

In [ ]:
filas = db.query("select * from tabla1")

print('1:', filas)
print('2:', filas[0])
print('3:', filas[0]['first_name'])

## Update

In [ ]:
sql = """\
    UPDATE tabla1 SET first_name = 'Alex'
    where first_name = 'Alejandro';
    """

assert 'WHERE' in sql.upper()

db.query(sql)

rows = db.query("select * from tabla1")
for row in rows:
    print( row )

## Delete

In [ ]:
sql = """\
    DELETE FROM tabla1
    WHERE first_name = 'Elon';
    """

assert 'WHERE' in sql.upper()

db.query(sql)

rows = db.query("select * from tabla1")
for row in rows:
    print( row['first_name'] )

## Drop Table

In [ ]:
db.query("DROP TABLE tabla1")

print("deleted tabla1")

try:
    db.query("DROP TABLE tabla1")
except Exception:
    print("tabla1 doesn't exist")


## Más ejemplos de cadenas de conexión
___

Cada base de datos tiene su propia `.jar` JDBC y su cadena de conexión, en el ejemplo anterior eran `sqlite-jdbc-3.36.0.3.jar` (clase `org.sqlite.JDBC`) y `jdbc:sqlite:{path}` respectivamente. En internet se pueden encontrar los JDBC de otros tipos de BBDD. Aquí algunos ejemplos:

### Microsoft SQL Server
[url de descarga](https://docs.microsoft.com/es-es/sql/connect/jdbc/download-microsoft-jdbc-driver-for-sql-server)

dentro del zip hay que buscar el `mssql-jdbc-9.4.0.jre{local_java_version}.jar` donde `local_java_version` sería la versión del jdk de java instalada (por ejemplo jdk 1.8 -> `mssql-jdbc-9.4.0.jre8.jar`) y la variable de entorno JAVA_HOME apuntando ahí. El resto de archivos se pueden borrar

```python
db = Database("com.microsoft.sqlserver.jdbc.SQLServerDriver", 
              "jdbc:sqlserver://<server_name>[:portNumber];databaseName=<db_name>", ["<user>", "<pass>"], "mssql-jdbc-9.4.0.jre8.jar")
# si es por instancia
db = Database("com.microsoft.sqlserver.jdbc.SQLServerDriver", 
              "jdbc:sqlserver://<server_name>[\instanceName]", ["<user>", "<pass>"], "mssql-jdbc-9.4.0.jre8.jar")
```

### Oracle
[url de descarga](https://www.oracle.com/es/database/technologies/appdev/jdbc-downloads.html)

dentro del zip hay que buscar el `ojdbc6.jar`. El resto de archivos se pueden borrar

```python
db = Database("oracle.jdbc.driver.OracleDriver", "jdbc:oracle:thin:@<server>[:<1521>]:<database_name>", ["<user>", "<pass>"], "ojdbc6.jar") 
```

### Sybase
[url de descarga](https://sourceforge.net/projects/jtds/files/latest/download)

dentro del zip hay que buscar el `jtds-1.3.1.jar`. El resto de archivos se pueden borrar

```python
db = Database("net.sourceforge.jtds.jdbc.Driver", "jdbc:jtds:sybase://<hostname>[:<4100>]/<dbname>", ["<user>", "<pass>"], "jtds-1.3.1.jar")
```